<table align="left">
  <td>
    <a href="https://colab.research.google.com/github.com/phonchi/Fundamental-tools-for-data-science/blob/main/ref-imp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/phonchi/Fundamental-tools-for-data-science/blob/main/ref-imp.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/phonchi/Fundamental-tools-for-data-science/blob/main/ref-imp.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

version_fn = getattr(tf.keras, "version", None)
if version_fn and version_fn().startswith("3."):
    import tf_keras as keras
else:
    keras = tf.keras

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

2024-08-05 04:08:45.161965: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 04:08:45.162076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 04:08:45.287154: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=1

batch_size=64


train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
        "../input/data-science-capstone-project-spring-2022/training_data/training_data",
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training',
    color_mode='grayscale'
)

Found 27785 images belonging to 800 classes.


A couple utility functions to plot grayscale and RGB images:

In [4]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

## Data preprocessing



In [5]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    "../input/data-science-capstone-project-spring-2022/training_data/training_data",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=(50, 50),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    "../input/data-science-capstone-project-spring-2022/training_data/training_data",
    validation_split=0.2,
    seed=1337,
    subset="validation",
    image_size=(50, 50),
    batch_size=32)

Found 34428 files belonging to 800 classes.
Using 27543 files for training.
Found 34428 files belonging to 800 classes.
Using 6885 files for validation.


Let’s look at the output of one of these Dataset objects: it yields batches of `180 × 180` RGB images (shape `(32, 180, 180, 3)`) and integer labels (shape `(32,)`). There are 32 samples in each batch (the batch size).

In [6]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

data batch shape: (32, 50, 50, 3)
labels batch shape: (32,)


## Leveraging a pretrained model

In [7]:
def get_hub_url_and_isize(model_name, ckpt_type, hub_type):
  if ckpt_type == '1k':
    ckpt_type = ''  # json doesn't support empty string
  else:
    ckpt_type = '-' + ckpt_type  # add '-' as prefix
  
  hub_url_map = {
    'efficientnetv2-b0': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0/{hub_type}',
    'efficientnetv2-b1': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1/{hub_type}',
    'efficientnetv2-b2': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2/{hub_type}',
    'efficientnetv2-b3': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3/{hub_type}',
    'efficientnetv2-s':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s/{hub_type}',
    'efficientnetv2-m':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m/{hub_type}',
    'efficientnetv2-l':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l/{hub_type}',

    'efficientnetv2-b0-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k/{hub_type}',
    'efficientnetv2-b1-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k/{hub_type}',
    'efficientnetv2-b2-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k/{hub_type}',
    'efficientnetv2-b3-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k/{hub_type}',
    'efficientnetv2-s-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k/{hub_type}',
    'efficientnetv2-m-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k/{hub_type}',
    'efficientnetv2-l-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k/{hub_type}',
    'efficientnetv2-xl-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k/{hub_type}',

    'efficientnetv2-b0-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k-ft1k/{hub_type}',
    'efficientnetv2-b1-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k-ft1k/{hub_type}',
    'efficientnetv2-b2-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k-ft1k/{hub_type}',
    'efficientnetv2-b3-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k-ft1k/{hub_type}',
    'efficientnetv2-s-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k-ft1k/{hub_type}',
    'efficientnetv2-m-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k-ft1k/{hub_type}',
    'efficientnetv2-l-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k-ft1k/{hub_type}',
    'efficientnetv2-xl-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k-ft1k/{hub_type}',
      
    # efficientnetv1
    'efficientnet_b0': f'https://tfhub.dev/tensorflow/efficientnet/b0/{hub_type}/1',
    'efficientnet_b1': f'https://tfhub.dev/tensorflow/efficientnet/b1/{hub_type}/1',
    'efficientnet_b2': f'https://tfhub.dev/tensorflow/efficientnet/b2/{hub_type}/1',
    'efficientnet_b3': f'https://tfhub.dev/tensorflow/efficientnet/b3/{hub_type}/1',
    'efficientnet_b4': f'https://tfhub.dev/tensorflow/efficientnet/b4/{hub_type}/1',
    'efficientnet_b5': f'https://tfhub.dev/tensorflow/efficientnet/b5/{hub_type}/1',
    'efficientnet_b6': f'https://tfhub.dev/tensorflow/efficientnet/b6/{hub_type}/1',
    'efficientnet_b7': f'https://tfhub.dev/tensorflow/efficientnet/b7/{hub_type}/1',
  }
  
  image_size_map = {
    'efficientnetv2-b0': 224,
    'efficientnetv2-b1': 240,
    'efficientnetv2-b2': 260,
    'efficientnetv2-b3': 300,
    'efficientnetv2-s':  384,
    'efficientnetv2-m':  480,
    'efficientnetv2-l':  480,
    'efficientnetv2-xl':  512,
  
    'efficientnet_b0': 224,
    'efficientnet_b1': 240,
    'efficientnet_b2': 260,
    'efficientnet_b3': 300,
    'efficientnet_b4': 380,
    'efficientnet_b5': 456,
    'efficientnet_b6': 528,
    'efficientnet_b7': 600,
  }
  
  hub_url = hub_url_map.get(model_name + ckpt_type)
  image_size = image_size_map.get(model_name, 224)
  return hub_url, image_size

In [8]:
import tensorflow_hub as hub

model_name = 'efficientnetv2-b0' #@param {type:'string'}
ckpt_type = '1k'   # @param ['21k-ft1k', '1k']
hub_type = 'feature-vector' # @param ['classification', 'feature-vector']
hub_url, image_size = get_hub_url_and_isize(model_name, ckpt_type, hub_type)

tf.keras.backend.clear_session()

data_augmentation = keras.Sequential(
    [
      keras.layers.Resizing(224, 224),
      keras.layers.RandomRotation(0.1),
      keras.layers.RandomTranslation(0, 0.1),
      keras.layers.RandomTranslation(0.1, 0),
      keras.layers.RandomZoom(0.1),
      keras.layers.RandomCrop(200,200),
    ])

inputs = keras.Input(shape=(50, 50, 3))
x = data_augmentation(inputs)
#x = keras.layers.Rescaling(1./255)(x)
# Apply input value scaling.
x = hub.KerasLayer(hub_url, trainable=True)(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(800, activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
# import tensorflow_hub as hub
# from tensorflow.keras import layers

# tf.keras.backend.clear_session()

# data_augmentation = keras.Sequential(
#     [
#       layers.Resizing(224, 224),
#       layers.RandomRotation(0.1),
#       layers.RandomTranslation(0, 0.1),
#       layers.RandomTranslation(0.1, 0),
#       layers.RandomZoom(0.1),
#       layers.RandomCrop(200,200),
#     ])

# hub_url = f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0/feature-vector'
# feature_extractor_layer = hub.KerasLayer(hub_url, trainable=True)

# # Create the sequential model
# model = tf.keras.Sequential([
#     # Input layer
#     tf.keras.layers.InputLayer(input_shape=(50, 50, 3)),
    
#     # Rescaling layer
#     #tf.keras.layers.Rescaling(1./255),
    
#     # Data augmentation
#     data_augmentation,
    
#     # Feature extraction layer from TensorFlow Hub
#     tf.keras.layers.Lambda(lambda x: feature_extractor_layer(x)),

    
#     # Dropout layer
#     tf.keras.layers.Dropout(0.5),
    
#     # Output layer
#     tf.keras.layers.Dense(800, activation='softmax')
# ])

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50, 50, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 200, 200, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 1280)              5919312   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 800)               1024800   
                                                                 
Total params: 6944112 (26.49 MB)
Trainable params: 6883504 (26.26 MB)
Non-trainable params: 60608 (236.75 KB)
_________________________________________________________________


In [10]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5, verbose=1)

callbacksa = [
    keras.callbacks.ModelCheckpoint(
        filepath="ev2_fine_tuning_0420.keras",
        save_best_only=True,
        monitor="val_loss"), lr_scheduler
]

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
              metrics=["accuracy"])

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacksa)

Epoch 1/50


I0000 00:00:1722831121.111393     119 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


861/861 [==============================] - 147s 105ms/step - loss: 4.1823 - accuracy: 0.2071 - val_loss: 1.8426 - val_accuracy: 0.5394 - lr: 0.1000
Epoch 2/50
861/861 [==============================] - 86s 100ms/step - loss: 1.1151 - accuracy: 0.7037 - val_loss: 0.4503 - val_accuracy: 0.8799 - lr: 0.1000
Epoch 3/50
861/861 [==============================] - 86s 100ms/step - loss: 0.5578 - accuracy: 0.8451 - val_loss: 0.3470 - val_accuracy: 0.9050 - lr: 0.1000
Epoch 4/50
861/861 [==============================] - 86s 100ms/step - loss: 0.3822 - accuracy: 0.8923 - val_loss: 0.2949 - val_accuracy: 0.9176 - lr: 0.1000
Epoch 5/50
861/861 [==============================] - 86s 100ms/step - loss: 0.2933 - accuracy: 0.9169 - val_loss: 0.2585 - val_accuracy: 0.9268 - lr: 0.1000
Epoch 6/50
861/861 [==============================] - 86s 100ms/step - loss: 0.2339 - accuracy: 0.9338 - val_loss: 0.2280 - val_accuracy: 0.9355 - lr: 0.1000
Epoch 7/50
861/861 [==============================] - 86s 100m

In [ ]:
model.save("my_keras_model_0420.keras")